# OrientDB

```
 docker compose up
```

## Limitações:
* API de Python (pyorient) não está sendo mais mantido desde 2017, aprestando problemas de compatibilidade de protocolo com verões mais novas;
* O suporte ao Tinkerpop ainda não está maduro, está em um projeto separado, como prova de conceito. Necessidade de compilar o projeto.
    * https://orientdb.dev/docs/3.2.x/gettingstarted/Tutorial-Working-with-graphs.html
    * https://biditsadhukhan.in/blogs/gremlin-orientdb/
    * https://github.com/orientechnologies/orientdb-gremlin
    * `Apache TinkerPop 3 graph structure implementation for OrientDB. This started off as just a proof of concept, but thanks to a lot of help it's now in a really good shape and it has been officially adopted by the OrientDB team to be part of OrientDB v3.x and should eventually replace OrientDB's graphdb implementation that is still on TinkerPop 2.`


In [ ]:
# Desinstalar pyorient oficial (não está mais sendo mantido)
#! pip uninstall pyorient   --> Última versão da api em 2017! https://pypi.org/project/pyorient/#history

# Instalar fork "atualizado":
! pip install git+https://github.com/OpenConjecture/pyorient.git

A API Python dá problema de protocolo (38).. ai existem alternativas como https://medium.com/@smelomedina05/establish-a-connection-between-orientdb-container-and-python-flask-3080b092196f para forcar um protocolo... 

In [ ]:
import requests
import json

class OrientDBRestClient:
    def __init__(self, host="localhost", port=2480, username="root", password="123456"):
        self.base_url = f"http://{host}:{port}"
        self.auth = (username, password)
        
    def create_database(self, db_name, db_type="graph"):
        """Criar banco via REST"""
        url = f"{self.base_url}/database/{db_name}/plocal/{db_type}"
        response = requests.post(url, auth=self.auth)
        
        if response.status_code in [200, 500]:  # 500 se já existe
            print(f"Banco '{db_name}' disponível")
            return True
        return False
    
    def execute_sql(self, db_name, sql_command):
        """Executar SQL via REST"""
        url = f"{self.base_url}/command/{db_name}/sql"
        
        response = requests.post(
            url,
            data=sql_command,
            headers={'Content-Type': 'text/plain'},
            auth=self.auth
        )
        
        if response.status_code == 200:
            try:
                return response.json()
            except:
                return {"result": "success"}
        else:
            print(f"Erro: {response.status_code} - {response.text}")
            return None


client = OrientDBRestClient()
db_name = "test_graph"

# 1. Criar banco
client.create_database(db_name)

# 2. Criar classes (sem usar métodos específicos do PyOrient)
client.execute_sql(db_name, "CREATE CLASS Person EXTENDS V")
client.execute_sql(db_name, "CREATE CLASS Knows EXTENDS E")

# 3. Inserir dados
client.execute_sql(db_name, "INSERT INTO Person SET name = 'Alice', age = 30")
client.execute_sql(db_name, "INSERT INTO Person SET name = 'Bob', age = 25")

# 4. Criar relacionamento
client.execute_sql(db_name, """
    CREATE EDGE Knows 
    FROM (SELECT FROM Person WHERE name = 'Alice') 
    TO (SELECT FROM Person WHERE name = 'Bob')
""")

# 5. Consultar
result = client.execute_sql(db_name, "SELECT * FROM Person")
if result and 'result' in result:
    print("Pessoas encontradas:")
    for person in result['result']:
        print(f"  - {person.get('name')}, {person.get('age')} anos")
